In [1]:
import lightgbm as lgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error


In [2]:
df_train = pd.read_csv("processed/processed_train_2.csv")
df_test = pd.read_csv("processed/processed_test_2.csv")
df_train

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan,price_doc
0,43,27.000000,4.0,12.559171,1879,1.909844,6.399244,2.105145,6.407578e+06,155572,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.668660e+06
1,34,19.000000,3.0,12.559171,1879,1.909844,6.399244,2.105145,9.589337e+06,115352,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.814010e+06
2,43,29.000000,2.0,12.559171,1879,1.909844,6.399244,2.105145,4.808270e+06,101708,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.523310e+06
3,89,50.000000,9.0,12.559171,1879,1.909844,6.399244,2.105145,1.258354e+07,178473,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.269391e+07
4,77,77.000000,4.0,12.559171,1879,1.909844,6.399244,2.105145,8.398461e+06,108171,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.582519e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30464,44,27.000000,7.0,9.000000,1975,2.000000,6.000000,3.000000,1.005305e+07,175518,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.170610e+06
30465,86,59.000000,3.0,9.000000,1935,4.000000,10.000000,3.000000,7.307411e+06,75377,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.422501e+07
30466,45,34.404467,10.0,20.000000,1879,1.000000,1.000000,1.000000,2.553630e+07,4001,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.754869e+06
30467,64,32.000000,5.0,15.000000,2003,2.000000,11.000000,2.000000,6.050065e+06,78616,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.308151e+07


In [3]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns=['price_doc']),
                                                 df_train['price_doc'], test_size=0.2, random_state=42)

In [4]:
## https://neptune.ai/blog/lightgbm-parameters-guide

In [5]:
from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "n_estimators": trial.suggest_int("n_estimators", 600, 1000),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0, 1),
        "bagging_freq": 1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
#         "max_bin": trial.suggest_int("min_data_in_leaf", 128, 512), 
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-10-31 23:46:19,321] A new study created in memory with name: no-name-31188165-b9ef-40a2-9a69-cd4017794e9f
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "min_data_in_leaf"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 100}
  warnings.warn(
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.7120774182829188, colsample_bytree=0.056934028582603824 will be ignored. Current value: feature_fraction=0.7120774182829188
[LightGBM] [Warning] min_data_in_leaf is set=91, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=91
[LightGBM] [Warning] bagging_fraction is set=0.6609260239396396, subsample=0.7164299078572999 will be ignored. Current value: bagging_fraction=0.6609260239396396
[LightGBM] [Warning] lambda_l2 is set=4.549772008900098, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.549772008900098
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 23:46:24,710] Trial 0 finished with value: 2863400.6909811897 and parameters: {'n_estimators': 884, 'boosting': 'rf', 'lambda_l2': 4.549772008900098, 'bagging_fraction': 0.6609260239396396, 'num_leaves': 369, 'feature_fraction': 0.7120774182829188, 'max_depth': 8, 'learning_rate': 0.00946101777543181, 'subsample': 0.7164299078572999, 'colsample_bytree': 0.056934028582603824, 'min_data_in_leaf': 91}. Best is trial 0 with value: 2863400.6909811897.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "min_data_in_leaf"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 100}
  warnings.warn(
C:\Users\Andrew\anaconda3\

[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] feature_fraction is set=0.6040203111978999, colsample_bytree=0.3898370022175877 will be ignored. Current value: feature_fraction=0.6040203111978999
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] bagging_fraction is set=0.7210517631410952, subsample=0.9402151401080291 will be ignored. Current value: bagging_fraction=0.7210517631410952
[LightGBM] [Warning] lambda_l2 is set=6.080764956985289, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.080764956985289
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 23:46:31,463] Trial 1 finished with value: 2845374.8494213633 and parameters: {'n_estimators': 648, 'boosting': 'rf', 'lambda_l2': 6.080764956985289, 'bagging_fraction': 0.7210517631410952, 'num_leaves': 676, 'feature_fraction': 0.6040203111978999, 'max_depth': 24, 'learning_rate': 0.05700731900442083, 'subsample': 0.9402151401080291, 'colsample_bytree': 0.3898370022175877, 'min_data_in_leaf': 50}. Best is trial 1 with value: 2845374.8494213633.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "min_data_in_leaf"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 100}
  warnings.warn(
C:\Users\Andrew\anaconda3\e

...

[I 2023-10-31 23:53:59,049] Trial 47 finished with value: 2519907.242566575 and parameters: {'n_estimators': 772, 'boosting': 'gbdt', 'lambda_l2': 6.767121130851216, 'bagging_fraction': 0.5814535678607654, 'num_leaves': 453, 'feature_fraction': 0.9375877358593894, 'max_depth': 45, 'learning_rate': 0.01293251578625176, 'subsample': 0.3631421483445886, 'colsample_bytree': 0.7334319494394973, 'min_data_in_leaf': 45}. Best is trial 32 with value: 2515081.7474339763.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "min_data_in_leaf"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 100}
  warnings.warn(
C:\Users\Andrew\anaconda

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8853694514323546, colsample_bytree=0.7171966273556752 will be ignored. Current value: feature_fraction=0.8853694514323546
[LightGBM] [Warning] min_data_in_leaf is set=26, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=26
[LightGBM] [Warning] bagging_fraction is set=0.5794850195521745, subsample=0.35875146979540595 will be ignored. Current value: bagging_fraction=0.5794850195521745
[LightGBM] [Warning] lambda_l2 is set=4.546752854781143, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.546752854781143
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 23:54:16,787] Trial 48 finished with value: 2563895.2229857943 and parameters: {'n_estimators': 785, 'boosting': 'gbdt', 'lambda_l2': 4.546752854781143, 'bagging_fraction': 0.5794850195521745, 'num_leaves': 625, 'feature_fraction': 0.8853694514323546, 'max_depth': 45, 'learning_rate': 0.013446294642321199, 'subsample': 0.35875146979540595, 'colsample_bytree': 0.7171966273556752, 'min_data_in_leaf': 26}. Best is trial 32 with value: 2515081.7474339763.
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\optuna\trial\_trial.py:677: RuntimeWarning: Inconsistent parameter values for distribution with name "min_data_in_leaf"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 1, 'low': 10, 'high': 100}
  warnings.warn(
C:\Users\Andrew\anaco

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.935445572458924, colsample_bytree=0.6185137577726177 will be ignored. Current value: feature_fraction=0.935445572458924
[LightGBM] [Warning] min_data_in_leaf is set=37, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=37
[LightGBM] [Warning] bagging_fraction is set=0.470501202096104, subsample=0.4206566649992301 will be ignored. Current value: bagging_fraction=0.470501202096104
[LightGBM] [Warning] lambda_l2 is set=5.140712736705392, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.140712736705392
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-31 23:54:32,367] Trial 49 finished with value: 3077874.192933537 and parameters: {'n_estimators': 761, 'boosting': 'dart', 'lambda_l2': 5.140712736705392, 'bagging_fraction': 0.470501202096104, 'num_leaves': 491, 'feature_fraction': 0.935445572458924, 'max_depth': 25, 'learning_rate': 0.00992953211803417, 'subsample': 0.4206566649992301, 'colsample_bytree': 0.6185137577726177, 'min_data_in_leaf': 37}. Best is trial 32 with value: 2515081.7474339763.


In [7]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'n_estimators': 777, 'boosting': 'gbdt', 'lambda_l2': 5.580160779406434, 'bagging_fraction': 0.5316225412489028, 'num_leaves': 63, 'feature_fraction': 0.8834161837506106, 'max_depth': 44, 'learning_rate': 0.011089672956973792, 'subsample': 0.49566146601559297, 'colsample_bytree': 0.675113736504296, 'min_data_in_leaf': 44}
Best RMSE: 2515081.7474339763


In [8]:
X = df_train.drop(['price_doc'], axis=1)
y = df_train['price_doc']

In [9]:
params = study.best_params
params["objective"] = "regression"
params["metric"] = "rmse"
params["verbosity"] = -1

In [10]:
model = lgb.LGBMRegressor(**params)

model.fit(X, y)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8834161837506106, colsample_bytree=0.675113736504296 will be ignored. Current value: feature_fraction=0.8834161837506106
[LightGBM] [Warning] min_data_in_leaf is set=44, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=44
[LightGBM] [Warning] bagging_fraction is set=0.5316225412489028, subsample=0.49566146601559297 will be ignored. Current value: bagging_fraction=0.5316225412489028
[LightGBM] [Warning] lambda_l2 is set=5.580160779406434, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.580160779406434


LGBMRegressor(bagging_fraction=0.5316225412489028, boosting='gbdt',
              colsample_bytree=0.675113736504296,
              feature_fraction=0.8834161837506106, lambda_l2=5.580160779406434,
              learning_rate=0.011089672956973792, max_depth=44, metric='rmse',
              min_data_in_leaf=44, n_estimators=777, num_leaves=63,
              objective='regression', subsample=0.49566146601559297,
              verbosity=-1)

In [11]:
df_test_copy = df_test.copy()

df_test = df_test.drop(['id'], axis=1)

In [12]:
pred = model.predict(df_test)
prediction_df = pd.DataFrame({
    'id': df_test_copy['id'],
    'price_doc': pred
})
prediction_df

,id,price_doc
0,30474,5.446781e+06
1,30475,8.053563e+06
2,30476,5.218428e+06
3,30477,5.697789e+06
4,30478,4.972711e+06
...,...,...
7657,38131,8.018036e+06
7658,38132,4.971977e+06
7659,38133,4.879705e+06
7660,38134,5.381328e+06


In [13]:
prediction_df.to_csv('./output/lgbm_pred_seed.csv', index=False)